**Title**: Update Timestamp on UI with Acquisition timestamp  
**Date**:  04-03-2020  
**Description**:  
* This notebook is used to update the timestamp on the FW Web UI based on the scan timestamp
* Functions: 

1.   `get_scan_moco_label` : Identify the original scan label
2.   `modify_acq_name` : Modify Acquisition name based on the session timepoint
3.  `update_acq_label` : Update the acquisition label on the FW instances





# Install and import dependencies

In [0]:
# Install specific packages required for this notebook
!pip install flywheel-sdk

In [0]:
# Import packages
from getpass import getpass
import logging
import os
import pandas as pd
import datetime
import flywheel

In [0]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

# Flywheel API Key and Client

Get a API_KEY. More on this at in the Flywheel SDK doc [here](https://flywheel-io.gitlab.io/product/backend/sdk/branches/master/python/getting_started.html#api-key).

In [0]:
API_KEY = getpass('Enter API_KEY here: ')

Instantiate the Flywheel API client

In [0]:
fw = flywheel.Client(API_KEY or os.environ.get('FW_KEY'))

Show Flywheel logging information

In [0]:
log.info('You are now logged in as %s to %s', fw.get_current_user()['email'], fw.get_config()['site']['api_url'])

# Suggestions on how to structure your notebook

## Constants

In [0]:
project_id = '5e87a92c5201f10c5b44d8aa'

project = fw.get_project(project_id)

## Helper functions

In [0]:
def get_updated_timestamp(item):
  """Getting the updated timestamp

  Args:
        item (dict): Dictionary with the timestamp on UI and from the scan 

  Return:

        (datetime): Updated Timestamp
        
  """

   correctTmstmp = modify_time(item['timestamp'], item['series_time'])
   update_container_timestamp(item['acq_id'], correctTmstmp)  
   return correctTmstmp


def modify_time(timestamp, series_time):
  """Modify the timestamp

  Args:
        timestamp (datetime): Timestamp from the UI
        series_time (datetime): Timestamp from the scan

  Return:

        (datetime): Updated Timestamp
        
  """
    if timestamp.hour != series_time.hour:
        timestamp = timestamp.replace(hour=series_time.hour)
    
    if timestamp.minute != series_time.minute:
        timestamp = timestamp.replace(minute=series_time.minute)

    if timestamp.second != series_time.second:
        timestamp = timestamp.replace(second=series_time.second)

    return timestamp


def update_container_timestamp(acq_id, ts):
   """Update the acquisition label on the project container

  Args:
        acq_id (str) : Acquisition ID
        ts (datetime) : Updated Timestamp
        
  """
  acq = fw.get_acquisition(acq_id)
  acq.update({'timestamp': ts})

## Main script

In [0]:
df = pd.DataFrame()

for session in project.sessions.iter():
    for acquisition in session.acquisitions.iter():
        acquisition = acquisition.reload()
        for file in acquisition.files : 
          if file['type'] == 'dicom':
            item = {
              'series_number': file.info.get('SeriesNumber'),
              'acq_label': acquisition.label,
              'acq_id': acquisition.id, 
              'timestamp': acquisition.timestamp,
              'series_time': datetime.datetime.strptime(file.info.get('SeriesTime'), '%H%M%S.%f'), 
            }    
            item['updated_timestamp'] = get_updated_timestamp(item)
             
            # append to the table
            df = df.append(item, ignore_index=True)


## Display Updated Acquisition Info


In [0]:
# display all of the columns 
pd.options.display.max_columns = None

display(df.head(50))